In [1]:
import sys
sys.path.append('../src')

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression # type: ignore
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns

import pickle

from functions import *
from data_cleaning import DataCleaning

from betfairlightweight import StreamListener, APIClient
import os


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv('../data/turf.csv')

df.head(5)

,Course,Course Direction,Distance (y),Race Type,Class,Going,Race Date,Race Time,Racecard No.,Draw,Draw IV,Position,Horse,DTN,DTW,Jockey,Jockey Claim,Trainer,Age,Weight (pounds),Penalty,Weight Rank,DSLR,Equip,Sex Abbrev,Favourite,BF Decimal SP,"Won (1=Won, 0=Lost)","Place (1=Placed, 0=UnPlaced)",PRC Average,PRC Last Run,PRC 2nd Last Run,PRC 3rd Last Run,Pace Abbrev,Pace Rating,Pace Rating Rank,OR,OR Rank,Tissue Odds Text,Tissue Odds Decimal,Tissue Rating,Tissue Rating Rank,TIS+PFR,PFR,PFR Rank,Main,Main Rank,Main Stats,Main+Stats Rank,Stats,Stats Rank,Trn Stats,Trainer Stats Rank,Jky Stats,Jockey Stats Rank,TrnJky Stats,Trainer/Jky Stats Rank,Hrs Stats,Horse Stats Rank,UR1,UR1 Rank,UR2,UR2 Rank,UR3,UR3 Rank,UR4,UR4 Rank,HA Career Speed Rating,HA Career Speed Rating Rank,HA Last 1 Year Speed Rating,HA Last 1 Year Speed Rating Rank,MR Career Speed Rating,MR Career Speed Rating Rank,MR Last 1 Year Speed Rating,MR Last 1 Year Speed Rating Rank,MR Last 3 Runs Speed Rating,MR Last 3 Runs Speed Rating Rank,LTO Speed Rating,LTO Speed Rating Rank,2nd LTO Speed Rating,2nd LTO Speed Rating Rank,3rd LTO Speed Rating,3rd LTO Speed Rating Rank,4th LTOt Speed Rating,4th LTO Speed Rating Rank,Horse Time In Seconds,dyHA,dyLTO,dyAVG,dyAVG(x),dyMR,dyMR(x),dy2HA,Runs Before,Won Before,Distance In Yards LTO (Same Code),Official Rating LTO (Same Code),FLAT TURF Runs Before,NHF Wins Before,Distance To Winner LTO,User Rating 1 LTO,User Rating 2 LTO,User Rating 3 LTO,User Rating 4 LTO,Speed Rating LTO,PRC Average LTO,PRC Last Run LTO,PRC 2nd Last Run LTO,PRC 3rd Last Run LTO,Today's Going Wins,Today's Going Places,Today's Going Runs,Today's Going PRB,Today's Going PRB^2,Today's Going 50% SR,Today's Distance Wins,Today's Distance Places,Today's Distance Runs,Today's Distance PRB,Today's Distance PRB^2,Today's Distance 50% SR,Today's Class Wins,Today's Class Places,Today's Class Runs,Today's Class PRB,Today's Class PRB^2,Today's Class 50% SR,Today's Course Wins,Today's Course Places,Today's Course Runs,Today's Course PRB,Today's Course PRB^2,Today's Course 50% SR,Pace Abbreviation LTO (Same Code)
0,Ascot,RH,1100,Turf,A,Good,05/10/2024,13:50,1,5,1.05,6.0,Democracy Dilemma,7.0,16.049999,Jamie Spencer,0,Robert Cowell,4,132,0,1,14.0,NaN,g,NaN,6.79,0,0,94,90,97,94,L,12,1,106,1,9/2,5.5,36,3,80,44,3,334,3,663,3,329,4,83,4,86,5,82,6,78,3,215,3,NaN,1,NaN,1,NaN,1,81,2,81,2,65,5,75,1,79,1,81,2,79,3,75,1,67,2,64.669998,81.0,81.0,62.0,78.0,64.0,79.0,79.0,29,7,1134,106.0,20,0,1.65,147.0,NaN,NaN,NaN,81.0,94.0,97.0,94.0,91.0,1,0,3,57.41,43.31,66.67,7,5,24,67.26,53.97,70.83,1,0,6,48.46,32.13,50.00,0,0,1,21.74,4.73,0.00,L
1,Ascot,RH,1100,Turf,A,Good,05/10/2024,13:50,2,3,0.67,5.0,Go Athletico,6.0,9.050000,D Egan,0,Adrian McGuinness,6,129,0,2,6.0,NaN,g,NaN,10.38,0,0,88,84,86,94,P,4,3,104,2,9/2,5.5,35,4,64,29,5,317,4,635,5,318,5,78,6,76,6,88,3,76,4,103,5,NaN,1,NaN,1,NaN,1,78,5,71,6,72,2,70,4,70,4,65,6,70,5,71,4,0,4,62.919998,78.0,65.0,72.0,68.0,72.0,70.0,75.0,7,2,1320,104.0,7,0,3.35,93.0,NaN,NaN,NaN,65.0,93.0,86.0,94.0,98.0,1,1,2,92.86,86.73,50.00,1,2,4,73.23,60.65,25.00,2,3,7,71.84,61.09,28.57,0,0,0,NaN,NaN,NaN,P
2,Ascot,RH,1100,Turf,A,Good,05/10/2024,13:50,3,1,1.24,1.0,Rumstar,0.0,0.000000,Hector Crouch,0,Jonathan Portman,4,129,0,2,21.0,NaN,g,NaN,7.08,1,0,87,93,76,91,P,4,3,101,5,14/1,15.0,13,6,32,19,6,281,6,579,6,298,6,80,5,91,2,83,5,44,6,93,6,NaN,1,NaN,1,NaN,1,78,5,78,5,69,4,70,4,70,4,70,4,66,6,72,3,64,3,60.660000,78.0,70.0,61.0,69.0,66.0,70.0,76.0,19,3,1243,101.0,18,0,0.70,29.0,NaN,NaN,NaN,70.0,78.0,76.0,91.0,68.0,0,0,0,NaN,NaN,NaN,2,1,6,65.69,56.17,66.67,1,0,9,50.49,31.89,44.44,0,0,3,34.72,18.58,33.33,P
3,Ascot,RH,1100,Turf,A,Good,05/10/2024,13:50,4,4,1.43,2.0,Beautiful Diamond,2.0,2.000000,Clifford Lee,0,K R Burke,3,127,0,4,20.0,NaN,f,3.0,4.05,0,1,95,91,96,97,P,6,2,104,2,7/2,4.5,44,2,90,46,2,342,2,683,2,341,3,91,3,87,4,96,2,67,5,231,2,NaN,1,NaN,1,NaN,1,80,4,80,4,74,1,75,1,75,2,74,3,80,2,75,1,75,1,61.160000,80.0,74.0,72.0,76.0,74.0,75.0,78.0,9,3,1100,104.0,9,0,4.20,31.0

In [26]:
df['sp_rank'] = df.groupby(['Race Date', 'Race Time', 'Course'])['BF Decimal SP'].rank(method='first')

In [3]:
bins = [800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600,
        2800, 3000, 3200, 3400, 3600, 3800, 4000]

df['distance_bucket_yards'] = pd.cut(df['Distance (y)'], bins = bins,
                               labels=['800-1000', '1000-1200', '1200-1400', '1400-1600', '1600-1800', '1800-2000',
                                       '2000-2200', '2200-2400', '2400-2600', '2600-2800', '2800-3000',
                                       '3000-3200', '3200-3400', '3400-3600', '3600-3800', '3800-4000'])

In [4]:
df['odds_on'] = (df['BF Decimal SP'] < 2.0).astype(int)

## EDA

In [28]:
race_count = df.groupby(['Race Date', 'Race Time', 'Course']).ngroups

print(f'There is {race_count} races in the dataset.')

There is 7387 races in the dataset.


In [23]:
total_count = len(df)

print(f'There are {total_count} entries in the dataset.')

There are 65680 entries in the dataset.


In [29]:
beaten_favs = df[(df['Won (1=Won, 0=Lost)'] == 0) & (df['sp_rank'] == 1)]

count_beaten_favs = len(beaten_favs)

print(f'The number of favourites beaten is: {count_beaten_favs}')

The number of favourites beaten is: 4952


In [33]:
prop_beaten_favs = 1 - count_beaten_favs / race_count

print(f'The proportion of favourites that win is: {prop_beaten_favs:.2%}')

The proportion of favourites that win is: 32.96%


## odds on favourites

In [25]:
beaten_short_priced_favs = df[(df['Won (1=Won, 0=Lost)'] == 0) & (df['odds_on'] == 1)]

count_beaten = len(beaten_short_priced_favs)

print(f'The number of odds on favourites beaten is: {count_beaten}')

The number of odds on favourites beaten is: 378


In [6]:
beaten_short_priced_favs.head()

,Course,Course Direction,Distance (y),Race Type,Class,Going,Race Date,Race Time,Racecard No.,Draw,Draw IV,Position,Horse,DTN,DTW,Jockey,Jockey Claim,Trainer,Age,Weight (pounds),Penalty,Weight Rank,DSLR,Equip,Sex Abbrev,Favourite,BF Decimal SP,"Won (1=Won, 0=Lost)","Place (1=Placed, 0=UnPlaced)",PRC Average,PRC Last Run,PRC 2nd Last Run,PRC 3rd Last Run,Pace Abbrev,Pace Rating,Pace Rating Rank,OR,OR Rank,Tissue Odds Text,Tissue Odds Decimal,Tissue Rating,Tissue Rating Rank,TIS+PFR,PFR,PFR Rank,Main,Main Rank,Main Stats,Main+Stats Rank,Stats,Stats Rank,Trn Stats,Trainer Stats Rank,Jky Stats,Jockey Stats Rank,TrnJky Stats,Trainer/Jky Stats Rank,Hrs Stats,Horse Stats Rank,UR1,UR1 Rank,UR2,UR2 Rank,UR3,UR3 Rank,UR4,UR4 Rank,HA Career Speed Rating,HA Career Speed Rating Rank,HA Last 1 Year Speed Rating,HA Last 1 Year Speed Rating Rank,MR Career Speed Rating,MR Career Speed Rating Rank,MR Last 1 Year Speed Rating,MR Last 1 Year Speed Rating Rank,MR Last 3 Runs Speed Rating,MR Last 3 Runs Speed Rating Rank,LTO Speed Rating,LTO Speed Rating Rank,2nd LTO Speed Rating,2nd LTO Speed Rating Rank,3rd LTO Speed Rating,3rd LTO Speed Rating Rank,4th LTOt Speed Rating,4th LTO Speed Rating Rank,Horse Time In Seconds,dyHA,dyLTO,dyAVG,dyAVG(x),dyMR,dyMR(x),dy2HA,Runs Before,Won Before,Distance In Yards LTO (Same Code),Official Rating LTO (Same Code),FLAT TURF Runs Before,NHF Wins Before,Distance To Winner LTO,User Rating 1 LTO,User Rating 2 LTO,User Rating 3 LTO,User Rating 4 LTO,Speed Rating LTO,PRC Average LTO,PRC Last Run LTO,PRC 2nd Last Run LTO,PRC 3rd Last Run LTO,Today's Going Wins,Today's Going Places,Today's Going Runs,Today's Going PRB,Today's Going PRB^2,Today's Going 50% SR,Today's Distance Wins,Today's Distance Places,Today's Distance Runs,Today's Distance PRB,Today's Distance PRB^2,Today's Distance 50% SR,Today's Class Wins,Today's Class Places,Today's Class Runs,Today's Class PRB,Today's Class PRB^2,Today's Class 50% SR,Today's Course Wins,Today's Course Places,Today's Course Runs,Today's Course PRB,Today's Course PRB^2,Today's Course 50% SR,Pace Abbreviation LTO (Same Code),distance_bucket_yards,odds_on
175,Ascot,RH,1100,Turf,A,Good To Firm,20/06/2024,14:30,13,10,0.91,4.0,Whistlejacket,0.1,1.3,Ryan Moore,0,A P O'Brien,2,129,0,1,45.0,NaN,c,1.0,1.97,0,0,93,96,89,0,L,8,2,0,1,3/1,4.00,66,1,152,86,1,321,2,669,1,348,2,97,1,95,1,94,1,62,6,252,1,NaN,1,NaN,1,NaN,1,75,1,75,1,66,3,66,3,66,3,75,1,58,4,0,2,0,1,60.090000,75.0,75.0,66.0,66.0,66.0,66.0,58.0,2,1,1100,0.0,2,0,0.00,158.0,NaN,NaN,NaN,75.0,89.0,89.0,0.0,0.0,0,0,0,NaN,NaN,NaN,1,0,1,100.00,100.00,100.00,1,0,1,100.0,100.00,100.00,0,0,0,NaN,NaN,NaN,L,1000-1200,1
404,Ascot,RH,1100,Turf,C,Good,11/05/2024,15:10,4,3,0.67,3.0,Hallasan,2.0,2.2,William Buick,0,Charlie Appleby,2,133,0,1,25.0,NaN,c,1.0,1.59,0,1,70,70,0,0,P,2,1,0,1,4/5,1.80,18,1,74,56,1,374,1,650,6,276,9,94,1,100,1,90,2,-8,9,315,1,NaN,1,NaN,1,NaN,1,50,1,50,1,50,1,50,1,50,1,50,1,0,1,0,1,0,1,61.250000,50.0,50.0,50.0,50.0,50.0,50.0,NaN,1,0,1100,0.0,1,0,1.05,304.0,NaN,NaN,NaN,50.0,0.0,0.0,0.0,0.0,0,0,1,60.00,36.00,0.00,0,0,1,60.00,36.00,0.00,0,0,0,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,P,1000-1200,1
874,Ascot,RH,1320,Turf,C,Good To Firm,11/07/2025,14:35,1,1,1.76,2.0,Front Line Fury,0.2,0.2,Callum Shepherd,0,George Scott,2,140,6,1,8.0,NaN,g,1.0,1.74,0,0,84,90,88,73,L,12,1,88,1,15/8,2.88,48,1,119,71,1,364,1,750,1,386,1,98,1,94,3,96,1,98,1,443,1,NaN,1,NaN,1,NaN,1,67,1,67,1,58,2,58,2,59,2,67,1,59,3,57,2,49,2,74.470001,67.0,67.0,58.0,61.0,58.0,59.0,59.0,4,2,1320,82.0,4,0,0.00,417.0,NaN,NaN,NaN,67.0,83.0,88.0,73.0,89.0,0,1,1,75.00,56.25,0.00,2,1,4,86.25,76.31,50.00,0,0,0,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,L,1200-1400,1
2584,Ascot,RH,2631,Turf,C,Good,06/10/2023,14:47,7,6,1.40,2.0,Scenic,5.5,5.5,Saffie Osborne,0,Ed Walker,3,125,0,6,21.0,NaN,f,1.0,1.80,0,1,89,85,87,94,H,6,1,95,1,6/5,2.20,62,1,101,39,2,359,1,668,2,309,8,86,2,81,4,84,3,58,8,352,1,NaN,1,NaN,1,NaN,1,80,1,80,1,58,3,65,2,74,1,56,3,74,1,80,1,53,1,158.029999,80.0,56.0,63.0,70.0,59.0,74.0,74.0,7,0,218

In [7]:
beaten_short_priced_favs['DSLR'].mean()

44.540389972144844

In [8]:
beaten_short_priced_favs['Pace Rating Rank'].mean()

2.574074074074074

In [9]:
beaten_short_priced_favs['PRC Last Run'].mean()

69.42592592592592

In [10]:
beaten_short_priced_favs['Main'].mean()

340.63492063492066

In [11]:
beaten_short_priced_favs['Tissue Rating Rank'].mean()

2.3624338624338623

In [12]:
beaten_short_priced_favs['Class'].value_counts(normalize=True)

Class
E    0.367725
D    0.251323
F    0.211640
A    0.082011
B    0.047619
C    0.039683
Name: proportion, dtype: float64

In [13]:
beaten_short_priced_favs['distance_bucket_yards'].value_counts()

distance_bucket_yards
1200-1400    77
1400-1600    67
1600-1800    65
1000-1200    42
2000-2200    30
1800-2000    27
2200-2400    21
2600-2800    19
3400-3600    12
2400-2600    10
3000-3200     8
800-1000      0
2800-3000     0
3200-3400     0
3600-3800     0
3800-4000     0
Name: count, dtype: int64

In [14]:
beaten_short_priced_favs['distance_bucket_yards'].value_counts(normalize=True)


distance_bucket_yards
1200-1400    0.203704
1400-1600    0.177249
1600-1800    0.171958
1000-1200    0.111111
2000-2200    0.079365
1800-2000    0.071429
2200-2400    0.055556
2600-2800    0.050265
3400-3600    0.031746
2400-2600    0.026455
3000-3200    0.021164
800-1000     0.000000
2800-3000    0.000000
3200-3400    0.000000
3600-3800    0.000000
3800-4000    0.000000
Name: proportion, dtype: float64

In [15]:
beaten_short_priced_favs['Course'].value_counts()

Course
Redcar         75
Salisbury      21
Windsor        20
Newmarket      19
Yarmouth       14
Brighton       14
Catterick      14
Leicester      13
Lingfield      11
Haydock        11
Hamilton       11
Doncaster      11
Sandown        11
Bath           11
Ripon          10
Musselburgh    10
Nottingham     10
Chepstow        9
Pontefract      9
York            8
Ffos Las        8
Epsom           8
Beverley        8
Chester         7
Carlisle        7
Newbury         6
Ayr             6
Thirsk          6
Ascot           5
Goodwood        4
Wetherby        1
Name: count, dtype: int64

In [16]:
beaten_short_priced_favs['Course'].value_counts(normalize=True)

Course
Redcar         0.198413
Salisbury      0.055556
Windsor        0.052910
Newmarket      0.050265
Yarmouth       0.037037
Brighton       0.037037
Catterick      0.037037
Leicester      0.034392
Lingfield      0.029101
Haydock        0.029101
Hamilton       0.029101
Doncaster      0.029101
Sandown        0.029101
Bath           0.029101
Ripon          0.026455
Musselburgh    0.026455
Nottingham     0.026455
Chepstow       0.023810
Pontefract     0.023810
York           0.021164
Ffos Las       0.021164
Epsom          0.021164
Beverley       0.021164
Chester        0.018519
Carlisle       0.018519
Newbury        0.015873
Ayr            0.015873
Thirsk         0.015873
Ascot          0.013228
Goodwood       0.010582
Wetherby       0.002646
Name: proportion, dtype: float64